In [0]:
configs ={"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id":"{Client ID}",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="{keyvault name}",key="{client secret name}"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{Directory ID}/oauth2/token" }
try:
    dbutils.fs.mount(
    source="abfss://data@{Storage Account Name}.dfs.core.windows.net/",
    mount_point="/mnt/data/",
    extra_configs = configs)
except Exception as e:
    print ("Error: {} already mounted.Run unmount first")

**Replace the Client ID, Key Vault Name, Client Secret Name, Directory ID, Storage Account Name in the above code with yours.**
    
    >If you get an error message saying the data is already mounted,
    unmount the data using the command **dbutils.fs.unmount("/mnt/data")**.

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

#create streaming delta live table
salesorderstreampath ='/mnt/data/'
df = spark.read.json(salesorderstreampath)
@dlt.table(
    comment="the streaming raw dataset."
    )
    
def sales_orders_stream_raw():
    return spark.readStream.schema(df.schema).json(salesorderstreampath)


In [0]:
# customers raw delta live table 
customerpath='/mnt/data/customers/customers.parquet'
@dlt.table(
    comment="the customers raw dataset."
    )
def customers_raw():
    return (spark.read.parquet(customerpath,header=True))

In [0]:
#products raw delta live table
productpath ='/mnt/data/products/products.parquet' 
@dlt.table(
    comment="the prdduct raw dataset."
    )
def products_raw():
    return (spark.read.parquet(productpath,header=True))

In [0]:
# sales order batch raw delta live table
salesorderbatchpath='/mnt/data/sales_orders/sales_orders.parquet' 
@dlt.table(
    comment="the sales order batch raw dataset.."
    )
def sales_orders_batch_raw():
    return (spark.read.parquet(salesorderbatchpath,header=True))